[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales (Examen final Jorge Carlos Pérez Barja)

En nuestro caso nos toco un dataset acerca de películas y debemos predecir las notas que tienen un grupo de películas en una página denominada IMDb.

·        El modelo debe ser construido utilizando el framework Pytorch.

·        Se debe realizar y explicar el preprocesamiento respectivo si así se requiriese.

·        El estudiante debe elegir el tipo de modelo (red o perceptrón), la infraestructura, los parámetros e hiperparámetros, y justificar detalladamente su elección y/o decisión.

·        Se deben desarrollar todas las métricas que respalden el punto anterior.

·        Se debe lograr la precisión y F1 score máximo.

In [2]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [3]:
D_in, H, D_out =35 , 20, 8

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
print(model)

Sequential(
  (0): Linear(in_features=35, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
)


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [ ]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [ ]:
#model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [ ]:
# used for manipulating directory paths
import os
import sklearn
# Scientific and vector computation for python
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

((70000, 784), (70000,))

In [ ]:
data = np.loadtxt("/content/drive/MyDrive/Inteligencia Artificial/Machine learning/Datasets/MACHINE_LEARNING_FINAL.csv",skiprows=1, delimiter='\|\|',dtype=bytes, usecols=[0]).astype(str)
datos = np.loadtxt(data, delimiter=';', usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36])
X = np.loadtxt(data, delimiter=';', usecols=[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36])
print(datos.shape)
#print(datos)
Y=datos[:,2]
X.shape, Y.shape

In [ ]:

# # normalización y split
# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)
X_train, X_test, y_train, y_test = X[:3000] , X[3000:] , Y[:3000].astype(np.int), Y[3000:].astype(np.int)
X_train.shape , X_test.shape

y_train.shape , y_test.shape

In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
# convertimos datos a tensores y copiamos en gpu
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 1.76380
Epoch 20/100 Loss 1.54751
Epoch 30/100 Loss 1.24620
Epoch 40/100 Loss 1.06159
Epoch 50/100 Loss 0.93045
Epoch 60/100 Loss 0.84029
Epoch 70/100 Loss 0.76547
Epoch 80/100 Loss 0.70578
Epoch 90/100 Loss 0.65771
Epoch 100/100 Loss 0.61810


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9291

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente. 

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.64787
Epoch 20/100 Loss 1.52765
Epoch 30/100 Loss 1.25656
Epoch 40/100 Loss 1.08139
Epoch 50/100 Loss 0.95054
Epoch 60/100 Loss 0.84715
Epoch 70/100 Loss 0.77140
Epoch 80/100 Loss 0.71515
Epoch 90/100 Loss 0.66624
Epoch 100/100 Loss 0.62576


0.9291

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class Model(torch.nn.Module):
    
    # constructor
    def __init__(self, D_in, H1, H2, D_out):
        
        # llamamos al constructor de la clase madre
        super(Model, self).__init__()
        
        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H1)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H1, H2)
        self.relu2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(H2, D_out)
        
    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones: 

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [ ]:
model = Model(35, 50, 100, 8)
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [ ]:
model

Model(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.08)

epochs = 1500
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/1500 Loss 0.31638
Epoch 20/1500 Loss 0.31542
Epoch 30/1500 Loss 0.31448
Epoch 40/1500 Loss 0.31355
Epoch 50/1500 Loss 0.31264
Epoch 60/1500 Loss 0.31175
Epoch 70/1500 Loss 0.31087
Epoch 80/1500 Loss 0.31000
Epoch 90/1500 Loss 0.30914
Epoch 100/1500 Loss 0.30829
Epoch 110/1500 Loss 0.30746
Epoch 120/1500 Loss 0.30664
Epoch 130/1500 Loss 0.30583
Epoch 140/1500 Loss 0.30502
Epoch 150/1500 Loss 0.30423
Epoch 160/1500 Loss 0.30344
Epoch 170/1500 Loss 0.30266
Epoch 180/1500 Loss 0.30190
Epoch 190/1500 Loss 0.30113
Epoch 200/1500 Loss 0.30038
Epoch 210/1500 Loss 0.29963
Epoch 220/1500 Loss 0.29889
Epoch 230/1500 Loss 0.29816
Epoch 240/1500 Loss 0.29744
Epoch 250/1500 Loss 0.29672
Epoch 260/1500 Loss 0.29600
Epoch 270/1500 Loss 0.29530
Epoch 280/1500 Loss 0.29460
Epoch 290/1500 Loss 0.29390
Epoch 300/1500 Loss 0.29321
Epoch 310/1500 Loss 0.29253
Epoch 320/1500 Loss 0.29185
Epoch 330/1500 Loss 0.29118
Epoch 340/1500 Loss 0.29051
Epoch 350/1500 Loss 0.28984
Epoch 360/1500 Loss 0.28918
E

0.9478

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [ ]:
class Model(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):        
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [ ]:
model = Model(784, 100, 10).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.55528
Epoch 20/100 Loss 1.13870
Epoch 30/100 Loss 0.93603
Epoch 40/100 Loss 0.82037
Epoch 50/100 Loss 0.75428
Epoch 60/100 Loss 0.69445
Epoch 70/100 Loss 0.64970
Epoch 80/100 Loss 0.61459
Epoch 90/100 Loss 0.58544
Epoch 100/100 Loss 0.56093


0.91

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [ ]:
model

Model(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
model.fc1

Linear(in_features=784, out_features=50, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[-0.0318,  0.0348,  0.0067,  ..., -0.0189, -0.0166,  0.0245],
        [ 0.0069,  0.0238,  0.0217,  ...,  0.0002,  0.0254,  0.0313],
        [ 0.0177, -0.0152, -0.0256,  ..., -0.0098,  0.0230,  0.0356],
        ...,
        [-0.0115, -0.0255, -0.0011,  ..., -0.0235, -0.0186,  0.0217],
        [ 0.0087,  0.0252, -0.0192,  ..., -0.0073,  0.0108, -0.0010],
        [ 0.0024, -0.0287, -0.0208,  ...,  0.0235,  0.0045, -0.0122]],
       requires_grad=True)

In [ ]:
model.fc1.bias

Parameter containing:
tensor([-4.8222e-03, -1.3689e-02,  2.1885e-02, -1.3090e-03, -3.2483e-02,
        -3.5488e-02,  3.6206e-03, -5.6117e-04, -2.2836e-02,  8.2669e-03,
         2.7917e-02, -1.0974e-02,  1.9112e-02, -2.3528e-02, -1.1465e-02,
        -2.1191e-02, -8.1123e-03, -1.9700e-02,  1.6371e-02,  3.0902e-02,
        -1.6740e-02,  2.6899e-02, -1.8657e-02, -2.5458e-02, -1.9838e-02,
        -8.5446e-03,  2.2225e-02, -2.6362e-03, -3.2634e-02,  2.9177e-02,
         6.9153e-03, -1.2329e-02, -1.0353e-02, -3.1109e-02, -3.4727e-02,
         7.8452e-03, -1.8916e-02, -6.5080e-03,  1.1855e-02,  1.6306e-02,
         6.0282e-05, -6.1625e-03, -3.1931e-02, -3.3686e-02,  1.5443e-02,
         2.5583e-02, -2.8966e-02, -7.9172e-03, -1.3064e-02,  3.0768e-02],
       requires_grad=True)

Es posible sobreescribir una capa de la siguiente manera

In [ ]:
model.fc1 = torch.nn.Linear(784, 100)
model.fc2 = torch.nn.Linear(100, 100)

model

Model(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [ ]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=10, bias=True)]

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
)

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes. 